## 데이터 베이스 구축

In [6]:
# 필요한 라이브러리

import pymysql
import pandas as pd

In [7]:
# MySQL 데이터 베이스 생성 함수

def create_database(database_name:str, username = 'root', password = '1234', host = 'localhost'):
    '''
    원하는 데이터베이스 이름을 문자열로 해서 함수 인수로 넣어주면 됩니다.
    그리고 password 인수 값에는 각자의 MySQL 패스워드 값으로 바꿔주세요.
    '''
    
    connection = pymysql.connect(
    host = host,
    user = username, 
    password = password)
    
    with connection.cursor() as cursor:
        cursor.execute(f"create database {database_name}")
    connection.commit()
    connection.close()

In [9]:
# 데이터 베이스 생성 함수 실행

database_name = "first_project"

create_database(database_name)

In [13]:
# 데이터 베이스 접근 함수

def get_connection(database_name, username = 'root', password = '1234', host = 'localhost'):
    conn = pymysql.connect(
        host = host,
        user = username,
        password = password,
        database = database_name,
        charset = 'utf8mb4',
        cursorclass = pymysql.cursors.DictCursor
    )
    return conn

In [14]:
# 연도 테이블 생성 함수

def year_table(update:list=None):
    '''
    처음에 2019~2023년의 연도 테이블을 만든다.
    그리고 나중에 이후 연도를 추가로 넣고 싶으면,
    update 인수에 리스트를 넣어준다.
    '''
    
    if update == None:
        year_values = list(range(2019, 2024))
    
        connection = get_connection(database_name)
        with connection.cursor() as cursor:
            cursor.execute("CREATE TABLE IF NOT EXISTS year (year_id INT AUTO_INCREMENT NOT NULL PRIMARY KEY, year INT NOT NULL)")
            for i in year_values:
                sql = f"INSERT INTO year (year) VALUES ({i})"   # INSERT INTO 테이블_이름 (열_이름) VALUES (값)
                cursor.execute(sql)

        connection.commit()
        connection.close()
    else:
        connection = get_connection(database_name)
        with connection.cursor() as cursor:
            for i in update:
                sql = f"INSERT INTO year (year) VALUES ({i})"
                cursor.execute(sql)

        connection.commit()
        connection.close()

In [15]:
# 연도 테이블 생성 함수 실행

year_table()

In [16]:
# 차종 테이블 생성 함수

def car_type():
    '''
    차종 테이블은 차종이 고정되어 있으므로, 데이터를 따로 받지 않는다.
    '''

    car_type = ['승용', '승합', '화물', '특수']

    connection = get_connection(database_name)
    with connection.cursor() as cursor:
        cursor.execute("CREATE TABLE IF NOT EXISTS car_type (car_id INT AUTO_INCREMENT NOT NULL PRIMARY KEY, type VARCHAR(2) NOT NULL)")
        for i in car_type:
            sql = f"INSERT INTO car_type (type) VALUES ('{i}')"
            cursor.execute(sql)
            
    connection.commit()
    connection.close()

In [17]:
# 차종 테이블 생성 함수 실행

car_type()

In [18]:
# 데이터 입력 함수

def Insert_data(filename:str, update:list=None):
    '''
    연도별 차종별 데이터가 함께 있는 파일의 이름을 문자열로 넣어 준다.
    초기 데이터 삽입 이후에 추가로 데이터를 넣고 싶다면,
    데이터 파일명에다가 연도 리스트 또한 update 인수를 통해 전달해준다.
    '''
    
    len_type = list(range(1,5))
    data = pd.read_csv(filename)

    connection = get_connection(database_name)
    with connection.cursor() as cursor:
        cursor.execute("CREATE TABLE IF NOT EXISTS car_amount (year_id INT NOT NULL, car_id INT NOT NULL, amount INT NOT NULL, FOREIGN KEY (year_id) REFERENCES year(year_id), FOREIGN KEY (car_id) REFERENCES car_type(car_id))")
    connection.commit()
    connection.close()
        
    connection = get_connection(database_name)
    with connection.cursor() as cursor:
        sql = "SELECT COUNT(year_id) AS total FROM year"
        cursor.execute(sql)
        result = cursor.fetchone()
        len_year = result['total'] + 1
        
        if update == None:
            year_list = list(range(1, len_year))
            for i in year_list:
                for j in len_type:
                    value = int(data.iloc[i-1,j])
                    sql = f"INSERT INTO car_amount (year_id, car_id, amount) VALUES ({i}, {j}, {value})"
                    cursor.execute(sql)
        else:
            year_table(update)
            new_year_list = list(range(len_year, len_year + len(update)))
            for i in new_year_list:
                for j in len_type:
                    value = int(data.iloc[i-len_year,j])
                    sql = f"INSERT INTO car_amount (year_id, car_id, amount) VALUES ({i}, {j}, {value})"
                    cursor.execute(sql)

    connection.commit()
    connection.close()

In [19]:
# 데이터 입력 함수 실행

Insert_data('car.csv')